# Keyword Research: Ultimate Google Suggest Script


Options for wildcard keyword suggestions:

1) given an query template.... google suggest all

2) given an input topic... use template and then google suggest all


---

### About Alton

Follow me for more data and tutorials

- twitter: https://twitter.com/alton_lex @alton_lex

- linkedin: https://www.linkedin.com/in/altonalexander/


### About Data Winners

Join the conversation:

- private Discord community

- Video tutorials

- Feedback and support on this and other scripts

Join now: https://datawinners.gumroad.com/l/data-analytics-for-seo

---

In [ ]:
# initialize Libraries

import requests
import json
import pandas as pd
import string

# Given one input template

Find all alphabetical and numerical recommendations

In [ ]:

topic_template = "seattle bike lanes *" #@param {type:"string"}


# Country and language to use
hl = 'us' # (us, uk , fr…)
gl = 'en' # Language to use (en, es, fr…)



all_suggestions = set()

# stop if google detects a bot
stop_bot_detected = False



# loop over each letter and number
options = [""]
options.extend(range(0,9))
options.extend(list(string.ascii_lowercase))

for char in options:
    
    if stop_bot_detected:
        continue
    
    # populate the template
    query = topic_template.replace("*", str(char)).replace("  "," ")
    
    print(query)
    url = "http://suggestqueries.google.com/complete/search?hl="+hl+"&gl="+gl+"&output=chrome&q=" + query

    response = requests.get(url, verify=True)

    if 'your computer or network may be sending automated queries' in response.text:
        stop_bot_detected = True
        continue
        
    suggestions = json.loads(response.text)

        
    for word in suggestions[1]:
        all_suggestions.add(word)


# output
print( "\nFound", str(len(all_suggestions)), "unique suggestions!" )

# save out the file
filename = "suggestions__"+topic_template.replace(" ","_")+'.txt'
with open(filename, 'w') as f:
    for line in list(all_suggestions):
        f.write(f"{line}\n")

# Given one input topic

Find all alphabetical and numerical recommendations

For a list of query templates

In [ ]:

topic_template = "seattle bike lanes *" #@param {type:"string"}

# Country and language to use
hl = 'us' # (us, uk , fr…)
gl = 'en' # Language to use (en, es, fr…)


# for recursive queries
query_templates = [
    "What is {topic} *",
    "Best {topic} *",
    "{topic} vs *",
    "How to * {topic}",
    "{topic} [alternatives] *",
    "common {topic} * issues",
    "common {topic} * mistakes",
    "common {topic} * benefits",
    "Types of * {topic}"
]



# reset
all_suggestions = set()
all_queries_complete = set()

# stop if google detects a bot
stop_bot_detected = False


# expand input topic into topics
for char in options:
    
    if stop_bot_detected:
        continue

    # populate the template
    query = topic_template.replace("*", str(char)).replace("  "," ")
    if query in all_queries_complete:
        continue
    all_queries_complete.add(query)
    
    print(query)
    url = "http://suggestqueries.google.com/complete/search?hl="+hl+"&gl="+gl+"&output=chrome&q=" + query

    response = requests.get(url, verify=True)

    if 'your computer or network may be sending automated queries' in response.text:
        stop_bot_detected = True
        continue
    
    suggestions = json.loads(response.text)

    for word in suggestions[1]:
        all_suggestions.add(word)
        
        
# use the first batch as the input for recursive set
all_topics = all_suggestions.copy()


# loop over each letter and number
options = [""]
options.extend(range(0,9))
options.extend(list(string.ascii_lowercase))



for this_topic in all_topics:
    for char in options:
        for query_template in query_templates:
            
            if stop_bot_detected:
                continue
            
            # populate the template
            query = query_template.format(topic=this_topic).replace("*", str(char)).replace("  "," ").strip()
            if query in all_queries_complete:
                continue
            all_queries_complete.add(query)

            print(query)
            url = "http://suggestqueries.google.com/complete/search?output=chrome&q=" + query

            response = requests.get(url, verify=True)

            if 'your computer or network may be sending automated queries' in response.text:
                stop_bot_detected = True
                continue
            suggestions = json.loads(response.text)

            for word in suggestions[1]:
                all_suggestions.add(word)

# Save the output

# output
print( "\nFound", str(len(all_suggestions)), "unique suggestions!" )

# save out the file
filename = "suggestions__"+topic_template.replace(" ","_")+'-recursive.txt'
with open(filename, 'w') as f:
    for line in list(all_suggestions):
        f.write(f"{line}\n")